# Scikit-Learn Linear Regression

### This implementation uses Scikit-Learn to build a linear regression model

## Goal: Predict used car prices and compare with No-Framework implementation

What Scikit-Learn provides (what we built manually in No-Framework):
- `train_test_split()`: Replaces our manual `train_test_split()` function
- `StandardScaler`: Replaces our manual `compute_scaling_params()` and `scale_features()` functions
- `LinearRegression`: Replaces our manual `foward()`, `compute_cost()`, `compute_gradients()`, and `train()` functions
- `mean_squared_error`, `r2_score`: Replaces our manual metric functions

Key Differerence: Scikit-Learn uses the **Normal Equation** (closed-form solution) instead of gradient descent. This means:
- No learning rate to tune
- No iterations to set
- Instant optimal solution (for datasets that fit in memory)



# Imports

In [1]:
# pandas: Data manipulation library for loading CSV files into dataframes
# We use pandas instead of numpy.genfromtxt() for easier column access
import pandas as pd

# numpy: Numerical operations library
# Still needed for array operations and setting random seed
import numpy as np

# matplotlib.pyplot: Plotting library for creating visualizations
import matplotlib.pyplot as plt

# os: Operating system interface for file path handling
import os

# Scikit-Learn Imports

# train_test_split: Splits array into random train and test subsets
from sklearn.model_selection import train_test_split

# StandardScaler: Standardizes features using z-score normalization
from sklearn.preprocessing import StandardScaler

# LinearRegression: Ordinary leas squares Linear Regression
# KEY DIFFERENCES: Uses normal Equation instead of gradient descent
from sklearn.linear_model import LinearRegression

# mean_squared_error: Calculates MSE between predictions and actual values
from sklearn.metrics import mean_squared_error, r2_score

# Set random seed for reproducabiility (matches our project-wide seed)
RANDOM_SEED = 113
np.random.seed(RANDOM_SEED)

print("All imports successful!")
print(f"Random seed set to: {RANDOM_SEED}")

All imports successful!
Random seed set to: 113


# Load cleaned data

- Load the same pre-processed dataset used in NF implementation
- This ensures a fair comparison between frameworks
- Using pandas instead of numpy.genfromtxt() for easier column handling

In [6]:
# Define path to our cleaned dataset
DATA_PATH = os.path.join('..', '..', 'data', 'processed', 'vehicles_clean.csv')

# pd.read_csv() reads a csv file into a pandas dataframe
# Unlike numpy.genfromtxt(), pandas:
# - Automatically detects and preserves column headers
# - Infers data types for each column
# - Provides easy column access via df['column_name']
# - Handles missing values gracefully
df = pd.read_csv(DATA_PATH)

# df.sahpe returns (rows, columns) as a tuple
print(f"Dataset shape: {df.shape}")

# df.column.tolist() converts column names to a python list
print(f"Columns: {df.columns.tolist()}")

# df.head(n) returns the first n rows as a DataFrame
# Useful for quick visual verification that data loaded correctly
print(f"\nFirst 3 rows:")
print(df.head(3))

Dataset shape: (100000, 12)
Columns: ['price', 'year', 'manufacturer', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status', 'transmission', 'drive', 'type', 'state']

First 3 rows:
   price    year  manufacturer  condition  cylinders  fuel  odometer  \
0  29990  2014.0             7          2          6     2   26129.0   
1   6995  2006.0            12          0          6     2  198947.0   
2   4995  2009.0            35          6          8     2  152794.0   

   title_status  transmission  drive  type  state  
0             0             2      0     8     17  
1             6             0      3    10      5  
2             0             0      3    11     22  


# Seperate Features and Target

- price (column 0) is our TARGET - what we want to predict
- All other columns (1-11) are FEATURES - inputs to our model
- Using pandas column selection instead of numpy array slicing

In [9]:
# Define which column is our target variable
TARGET_COLUMN = 'price'

# Define feature column names
FEATURE_COLUMNS = [ 'year', 'manufacturer', 'condition', 'cylinders',
                    'fuel', 'odometer', 'title_status', 'transmission', 'drive', 'type', 'state']

# Extract target variable (y) using pandas column selection
# df[column_name] returns a pandas Series (1D array with labels)
# .values converst it to a numpy array (matches NF format)
y = df[TARGET_COLUMN].values

# Extract features (X) using pandas column select
X = df[FEATURE_COLUMNS].values

# Verify shapes match No-Framework
# X should be (100000, 11)
# y should be (100000,)
print(f"Features (X) shape: {X.shape}")
print(f"Target (y) shape: {y.shape}")

# Store feature names for later use (displaying learned weights)
FEATURE_COLUMNS = FEATURE_COLUMNS
print(f"\nFeature Names: {FEATURE_COLUMNS}")

Features (X) shape: (100000, 11)
Target (y) shape: (100000,)

Feature Names: ['year', 'manufacturer', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status', 'transmission', 'drive', 'type', 'state']
